# Session Five 5 
# Programming with Python  MOD007891

# Understanding NumPy 1

## Session Outline



- Data in Python
- NumPy Basic
- NumPy Standard Data Types
- Array Slicing
- Reshaping of Arrays
- Universal Functions
- Aggregations



## Supplementary files:

- president_heights.csv
- Seattle2014.csv








 **Datasets** can come from a wide range of sources and a wide range of formats, including  collections of documents, collections of images, collections of sound clips, collections of numerical measurements, or nearly anything else.
 

Data can be converted to **arrays of numbers.** For example, images particularly **digital images** can be thought of as simply **two-dimensional arrays** of numbers representing pixel brightness across the area.


Sound clips can be thought of as **one-dimensional arrays of intensity versus time.**


**Text** can be converted in various ways into **numerical representations**, perhaps binary digits representing the frequency of certain words or pairs of words.


No matter what the data are, the first step in **making it analyzable** will be to transform them into **arrays of numbers.**


We need to find a way for **effectively loading, storing, and manipulating** arrays of numbers in Python.



And you guessed it....**NumPy** (short for *Numerical Python*) provides an efficient interface to store and operate on dense data buffers.


In some ways, NumPy arrays are like Python's built-in ``list`` type, but NumPy arrays provide much more **efficient storage** and **data operations** as the arrays grow larger in size.


NumPy arrays form the **core** of nearly the entire ecosystem of **data science** tools in Python.




In [2]:
import numpy
numpy.__version__

'1.20.3'

For the pieces of the package discussed here, I'd recommend **NumPy version 1.8 or later.**
By convention, you'll find that most people in the SciPy/PyData world will import NumPy using ``np`` as an alias:

In [3]:
import numpy as np

In [9]:
np?

Throughout this module you'll find that this is the way we will import and use NumPy.

## Reminder about Built In Documentation

As you read through this notebook, don't forget that IPython gives you the ability to quickly explore the contents of a package (by using the tab-completion feature), as well as the documentation of various functions (using the ``?`` character).

For example, to display all the contents of the numpy namespace, you can type this:

```ipython
In [3]: np.<TAB>
```

And to display NumPy's built-in documentation, you can use this:

```ipython
In [4]: np?
```

More detailed documentation, along with tutorials and other resources, can be found at http://www.numpy.org.

In [ ]:
np?

# Data in Python

## Why are Python native data types slow ? 


### Dynamic Typing


Users of Python are often drawn-in by its ease of use, one piece of which is **dynamic typing.**


While a **statically-typed** language like **C** or **Java** requires each variable to be **explicitly declared,** a dynamically-typed language like Python skips this specification. 



For example, in C you might specify a particular operation as follows:

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

While in Python the equivalent operation could be written this way:

```python
# Python code
result = 0
for i in range(100):
    result += i
```

Notice the main difference: in C, the data types of each variable are **explicitly declared**, while in Python the types are **dynamically inferred**. 

This means, for example, that we can assign **any kind of data** to any variable:

```python
# Python code
x = 4
x = "four"
```

Here we've switched the contents of ``x`` from an integer to a string. The same thing in C would lead (depending on compiler settings) to a compilation error or other unintented consequences:

```C
/* C code */
int x = 4;
x = "four";  // FAILS
```

This sort of flexibility is one piece that makes Python and other dynamically-typed languages convenient and easy to use.


Understanding *how* this works is an important piece of learning to **analyze data efficiently** and **effectively** with Python.


Type-flexibility **(Dynamic Typing)** in Python requires variables store more than just their value. They need to contain **extra information** about the **type** of the value.



## A Python Integer Is More Than Just an Integer

The standard Python implementation is written in C.

![Python Data Types](Images/cint_vs_pyint.png)




This means that every Python object is simply a **disguised C structure**, which contains **not only its value**, but **other information** as well. 

For example, when we define an integer in Python, such as ``x = 10000``, ``x`` is not just a "raw" integer. 

### It's actually a **pointer** to a **compound C structure**, which contains several values.


Looking through the Python 3.4 source code, we find that the integer (long) type definition effectively looks like this (once the C macros are expanded):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

A single integer in Python 3.4 actually contains four pieces:


- ``ob_refcnt``, a reference count that helps Python silently handle memory allocation and deallocation
- ``ob_type``, which encodes the type of the variable
- ``ob_size``, which specifies the size of the following data members
- ``ob_digit``, which contains the actual integer value that we expect the Python variable to represent.

This means that there is some overhead in storing an integer in Python as compared to an integer in a compiled language like C.



A Python integer is a **pointer to a position in memory** containing all the Python object information, including the bytes that contain the integer value.


This extra information in the Python integer structure is what allows Python to be coded so freely and dynamically.


All this additional information in Python types comes at a **cost**, however, which becomes especially apparent in structures that combine many of these objects.

## A Python List Is More Than Just a List

Let's consider now what happens when we use a Python **data structure that holds many Python objects.**


The standard mutable multi-element container in Python is the list.
We can create a list of integers as follows:

In [10]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
type(L[0])

int

Or, similarly, a list of strings:

In [12]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [13]:
type(L2[0])

str

Because of Python's **dynamic typing**, we can even create **heterogeneous lists:**

In [14]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

**But this flexibility comes at a cost:** to allow these flexible types, each item in the list must contain its own type info, reference count, and other information–that is, each item is a complete Python object.


In the special case that all variables are of the same type, **much of this information is redundant**: it can be much more efficient to store data in a fixed-type array.


The difference between a **dynamic-type list** and a **fixed-type (NumPy-style) array** is illustrated in the following figure:

![Array Memory Layout](Images/array_vs_list.png)

At the implementation level, the array essentially contains **a single pointer** to one **contiguous** block of data.


The Python list, on the other hand, **contains a pointer to a block of pointers**, each of which in turn points to a full Python object like the Python integer we saw earlier.


Again, the advantage of the list is flexibility: because each list element is a full structure containing both data and type information, the list can be filled with data of any desired type.


Fixed-type NumPy-style arrays lack this flexibility, but are much **more efficient for storing and manipulating data.**

## Fixed-Type Arrays in Python

Python offers several different options for storing data in **efficient, fixed-type data buffers.**


The built-in ``array`` module (available since Python 3.3) can be used to create dense arrays of a uniform type:

In [16]:
import array
L = list(range(10))
A = array.array('i', L)
A


array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [17]:
type(A)

array.array

Here ``'i'`` is a type code indicating the contents are **integers.**

Much more useful, however, is **the ``ndarray`` object of the NumPy package.**


While Python's ``array`` object provides efficient storage of array-based data, NumPy adds to this efficient *operations* on that data.


We will explore these operations in later sections; here we'll demonstrate several ways of creating a NumPy array.

We'll start with the standard NumPy import, under the alias ``np``:

In [ ]:
import numpy as np

## Creating Arrays from Python Lists

First, we can use ``np.array`` to create arrays from **Python lists:**

In [18]:
L=[1, 4, 2, 5, 3]

In [19]:
type(L)

list

In [20]:
# integer array:
x = np.array(L)
x

array([1, 4, 2, 5, 3])

In [21]:
type(np.array(L))

numpy.ndarray

In [22]:
x.dtype

dtype('int32')

Remember that **unlike Python lists, NumPy is constrained to arrays that all contain the same type.**


If types do not match, NumPy will **upcast if possible** (here, integers are up-cast to floating point):

In [26]:
ar = np.array([3.14, 4, 2, 3,'see'])

In [27]:
type(ar)

numpy.ndarray

In [28]:
ar.dtype

dtype('<U32')

In [ ]:
ar

If we want to **explicitly** set the data type of the resulting array, we can use the ``dtype`` keyword:

In [31]:
np.array([1, 2, 3, 4,], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

Finally, unlike Python lists, NumPy arrays can **explicitly be multi-dimensional**; 


Here's one way of initializing a multidimensional array using a list of lists:

In [32]:
# nested lists result in multi-dimensional arrays
# i=2 => range(2,5) => 2,3,4
# i=4 => range(4,7) => 4,5,6
# i=6 => range(6,8) => 6,7,8

x=np.array([range(i, i + 3) for i in [2, 4, 6]])
x

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

In [64]:
xx = np.array([range(i,i+2,2) for i in range(0,20,2)])
xx

array([[ 0],
       [ 2],
       [ 4],
       [ 6],
       [ 8],
       [10],
       [12],
       [14],
       [16],
       [18]])

In [65]:
# Python len method... Not Good here :(
len(x)

3

In [6]:
#numpy only method
np.size(x)

9

In [66]:
type(x)

numpy.ndarray

In [35]:
x.dtype

dtype('int32')

The inner lists are treated as rows of the resulting two-dimensional array.

## Creating Arrays from Scratch

Especially for larger arrays, it is more efficient to **create arrays from scratch using routines built into NumPy.**
Here are several examples:

In [40]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=str)

array(['', '', '', '', '', '', '', '', '', ''], dtype='<U1')

In [42]:
# Create a 3x5 floating-point array filled with ones
np.ones((3, 5,6), dtype=float)


array([[[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]]])

In [67]:
# Create a 3x5 array filled with 3.14
np.full((2, 5,2),7)

array([[[7, 7],
        [7, 7],
        [7, 7],
        [7, 7],
        [7, 7]],

       [[7, 7],
        [7, 7],
        [7, 7],
        [7, 7],
        [7, 7]]])

In [47]:
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20,3)
np.arange()

array([ 0,  3,  6,  9, 12, 15, 18])

In [49]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 2, 5)

array([0. , 0.5, 1. , 1.5, 2. ])

In [50]:
# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))

array([[0.56087454, 0.58458692, 0.56338757],
       [0.51225307, 0.21480613, 0.00290703],
       [0.05466431, 0.11843818, 0.09263797]])

In [51]:
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

array([[ 1.72726826,  1.12577565, -2.37203614],
       [-0.14698805,  1.21741597,  1.56159296],
       [ 0.9839364 ,  1.21975376, -1.19413895]])

In [54]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 20, (3, 4))

array([[18, 17, 17, 14],
       [11,  0, 12, 19],
       [ 0, 12,  8,  3]])

In [55]:
# Create a 3x3 identity matrix
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [3]:
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([3.56035923e-307, 1.37959129e-306, 1.78021798e-306])

## NumPy Standard Data Types

NumPy arrays contain values of a single type, so it is important to have detailed knowledge of those types and their limitations.
Because NumPy is built in C, the types will be familiar to users of C, Fortran, and other related languages.

The standard NumPy data types are listed in the following table.
Note that when constructing an array, they can be specified using a string:

```python
np.zeros(10, dtype='int16')
```

Or using the associated NumPy object:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 

# The Basics of NumPy Arrays

**Data manipulation** in Python is nearly synonymous with **NumPy array** manipulation.


This section will present several examples of using NumPy array manipulation to access data and subarrays, and to split, reshape, and join the arrays.


While the types of operations shown here may seem a bit dry and pedantic, they comprise the building blocks of many other examples used throughout this module.
Get to know them well!

We'll cover a few categories of basic array manipulations here:

- ***Attributes of arrays***: Determining the size, shape, memory consumption, and data types of arrays
- ***Indexing of arrays***: Getting and setting the value of individual array elements
- ***Slicing of arrays***: Getting and setting smaller subarrays within a larger array
- ***Reshaping of arrays***: Changing the shape of a given array
- ***Joining and splitting of arrays***: Combining multiple arrays into one, and splitting one array into many

## NumPy Array Attributes
First let's discuss some useful array attributes.
We'll start by defining three random arrays, a one-dimensional, two-dimensional, and three-dimensional array.
We'll use NumPy's random number generator, which we will **seed** with a set value in order to ensure that the same random arrays are generated each time this code is run:

In [71]:
import numpy as np
np.random.seed(0)  # seed for reproducibility

x1 = np.random.randint(10,size=6)  # One-dimensional array
x2 = np.random.randint(10, size=(3, 4))  # Two-dimensional array
x3 = np.random.randint(10, size=(3, 4, 5))  # Three-dimensional array

In [69]:
x1

array([5, 0, 3, 3, 7, 9])

In [23]:
x2

array([[3, 5, 2, 4],
       [7, 6, 8, 8],
       [1, 6, 7, 7]])

In [9]:
x3

array([[[8, 1, 5, 9, 8],
        [9, 4, 3, 0, 3],
        [5, 0, 2, 3, 8],
        [1, 3, 3, 3, 7]],

       [[0, 1, 9, 9, 0],
        [4, 7, 3, 2, 7],
        [2, 0, 0, 4, 5],
        [5, 6, 8, 4, 1]],

       [[4, 9, 8, 1, 1],
        [7, 9, 9, 3, 6],
        [7, 2, 0, 3, 5],
        [9, 4, 4, 6, 4]]])

Each array has attributes ``ndim`` (the number of dimensions), ``shape`` (the size of each dimension), and ``size`` (the total size of the array):

In [ ]:
print("x3 ndim: ", x3.ndim)
print("x3 shape:", x3.shape)
print("x3 size: ", x3.size)

Another useful attribute is the ``dtype``, the data type of the array (which we discussed previously. 

In [ ]:
print("dtype:", x3.dtype)

Other attributes include ``itemsize``, which lists the size **(in bytes)** of each array element, and ``nbytes``, which lists the **total size (in bytes)** of the array:

In [72]:
print("itemsize:", x3.itemsize, "bytes")
print("nbytes:", x3.nbytes, "bytes")

itemsize: 4 bytes
nbytes: 240 bytes


In general, we expect that ``nbytes`` is equal to ``itemsize`` times ``size``.

In [73]:
# nbytes, bytes and array's shape
4*(3*4*5)

240

## Array Indexing: Accessing Single Elements
If you are familiar with Python's standard list indexing, indexing in NumPy will feel quite familiar.
In a one-dimensional array, the $i^{th}$ value (counting from zero) can be accessed by specifying the desired index in square brackets, just as with Python lists:

In [ ]:
x1

In [ ]:
x1[0]

In [ ]:
x1[4]

In [ ]:
x1[-1]

In [ ]:
x1[-2]

In [21]:
x2

array([99, 99])

In [ ]:
x2[0, 0]

In [ ]:
x2[2, 0]

In [ ]:
x2[2, -1]

In [ ]:
x2[0, 0] = 12
x2

Keep in mind that, unlike Python lists, NumPy arrays have a **fixed type.**


This means, for example, that if you attempt to insert a floating-point value to an integer array, the value will be **silently truncated**. Don't be caught unaware by this behavior!

In [ ]:
x1[0] = 3.14159  # this will be truncated!
x1

## Array Slicing: Accessing Subarrays
Just as we can use **square brackets** to access individual array elements, we can also use them to access subarrays with the **slice** notation, marked by the colon (``:``) character.


The NumPy slicing syntax follows that of the **standard Python** list; to access a slice of an array ``x``, use this:
``` python
x[start:stop:step]
```
If any of these are unspecified, they default to the values ``start=0``, ``stop=``*``size of dimension``*, ``step=1``.
We'll take a look at accessing sub-arrays in one dimension and in multiple dimensions.

### One-dimensional subarrays

In [19]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
x[:5]  # first five elements

In [ ]:
x[5:]  # elements after index 5

In [ ]:
x[4:7]  # middle sub-array

In [ ]:
x[::2]  # every other element

A potentially confusing case is when the ``step`` value is negative.
In this case, the defaults for ``start`` and ``stop`` are swapped.
This becomes a convenient way to reverse an array:

In [ ]:
x[::-1]  # all elements, reversed

In [11]:
x = np.arange(10)
x[5::-2] 

array([5, 3, 1])

### Multi-dimensional subarrays

Multi-dimensional slices work in the same way, with multiple slices separated by commas.
For example:

In [20]:
x2

array([99, 99])

In [ ]:
x2[:2, :3]  # two rows, three columns

In [26]:
print(x2)

[[3 5 2 4]
 [7 6 8 8]
 [1 6 7 7]]


In [27]:
x2[:3, ::2]  

array([[3, 2],
       [7, 8],
       [1, 7]])

Finally, subarray dimensions can even be reversed together:

In [13]:
print(x2)
x2[::-1, ::-1]

[[3 5 2 4]
 [7 6 8 8]
 [1 6 7 7]]


array([[7, 7, 6, 1],
       [8, 8, 6, 7],
       [4, 2, 5, 3]])

#### Accessing array rows and columns

One commonly needed routine is accessing of single rows or columns of an array.
This can be done by combining indexing and slicing, using an empty slice marked by a single colon (``:``):

In [ ]:
x2

In [ ]:
print(x2[:, 0])  # first column of x2

In [ ]:
print(x2[0, :])  # first row of x2

In [ ]:
print(x2[0])  # equivalent to x2[0, :]

### Subarrays as no-copy views

One important–and extremely useful–thing to know about array slices is that they **return *views*** rather than **copies** of the array data.


This is one area in which NumPy array slicing differs from Python list slicing: in lists, slices will be copies.
Consider our two-dimensional array from before:

In [ ]:
print(x2)

In [14]:
x2_sub = x2[:2, :2]
print(x2_sub)

[[3 5]
 [7 6]]


In [ ]:
print(x2)

Now if we modify this subarray, we'll see that the original array is changed! Observe:

In [ ]:
x2_sub[0, 0] = 99
print(x2_sub)

In [ ]:
# shocked??? 
print(x2)

This default behavior is actually **quite useful**.


it means that when we work with large datasets, we can access and **process pieces of these datasets** without the need to copy the underlying data buffer.

Good for **divide and conquer data manipulation.**

### Creating copies of arrays

Despite the nice features of array views, it is sometimes useful to instead explicitly copy the data within an array or a subarray. This can be most easily done with the ``copy()`` method:

In [ ]:
x2_sub_copy = x2[:2, :2].copy()
print(x2_sub_copy)

If we now modify this subarray, the original array is not touched:

In [ ]:
x2_sub_copy[0, 0] = 42
print(x2_sub_copy)

In [ ]:
print(x2)

## Reshaping of Arrays

Another useful type of operation is reshaping of arrays.
The most flexible way of doing this is with the ``reshape`` method.


For example, if you want to put the numbers 1 through 9 in a $3 \times 3$ grid, you can do the following:

In [15]:
grid = np.arange(1, 10).reshape((3, 3))
print(grid)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


Note that for this to work, the **size of the initial array must match the size of the reshaped array. **


Where possible, the ``reshape`` method will use a **no-copy view** of the initial array, but with non-contiguous memory buffers this is not always the case.

Another common reshaping pattern is the conversion of a **one-dimensional** array into a **two-dimensional** row or column matrix.


This can be done with the ``reshape`` method, or more easily done by making use of the ``newaxis`` keyword within a slice operation:

In [ ]:
x = np.array([1, 2, 3])

# row vector via reshape
y= x.reshape((1, 3))
y

In [ ]:
y

In [ ]:
y.ndim

In [ ]:
y.shape

In [ ]:
# row vector via newaxis
x[np.newaxis, :]

In [ ]:
# column vector via reshape
x.reshape((3, 1))

In [ ]:
# column vector via newaxis
x[:, np.newaxis]

We will see this type of transformation often throughout the remainder of this module.

## Array Concatenation and Splitting

All of the preceding routines worked on single arrays. It's also possible to combine multiple arrays into one, and to conversely split a single array into multiple arrays. We'll take a look at those operations here.

### Concatenation of arrays

Concatenation, or joining of two arrays in NumPy, is primarily accomplished using the routines ``np.concatenate``, ``np.vstack``, and ``np.hstack``.


``np.concatenate`` takes a tuple or list of arrays as its first argument, as we can see here:

In [ ]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])

You can also concatenate more than two arrays at once:

In [ ]:
z = [99, 99, 99]
print(np.concatenate([x, y, z]))

It can also be used for two-dimensional arrays:

In [ ]:
grid = np.array([[1, 2, 3],
                 [4, 5, 6]])

In [ ]:
# concatenate along the first axis
np.concatenate([grid, grid])

In [ ]:
# concatenate along the second axis (zero-indexed)
np.concatenate([grid, grid], axis=1)

For working with arrays of mixed dimensions, it can be clearer to use the ``np.vstack`` (vertical stack) and ``np.hstack`` (horizontal stack) functions:

In [ ]:
x = np.array([1, 2, 3])
grid = np.array([[9, 8, 7],
                 [6, 5, 4]])

# vertically stack the arrays
np.vstack([x, grid])

In [ ]:
# horizontally stack the arrays
y = np.array([[99],
              [99]])
np.hstack([grid, y])

Similary, ``np.dstack`` will stack arrays along the third axis.

### Splitting of arrays

The opposite of concatenation is splitting, which is implemented by the functions ``np.split``, ``np.hsplit``, and ``np.vsplit``.  For each of these, we can pass a list of indices giving the split points:

In [16]:
x = [1, 2, 3, 99, 99, 3, 2, 1]
x1, x2, x3 = np.split(x, [3, 5])
print(x1, x2, x3)

[1 2 3] [99 99] [3 2 1]


Notice that *N* split-points, leads to *N + 1* subarrays.
The related functions ``np.hsplit`` and ``np.vsplit`` are similar:

In [ ]:
grid = np.arange(16).reshape((4, 4))
grid

In [ ]:
upper, lower = np.vsplit(grid, [2])
print(upper)
print(lower)

In [ ]:
left, right = np.hsplit(grid, [2])
print(left)
print(right)

Similarly, ``np.dsplit`` will split arrays along the third axis.

# Computation on NumPy Arrays: Universal Functions
Up until now, we have been discussing some of the basic nuts and bolts of NumPy; in the next few sections, we will dive into the reasons that NumPy is so important in the Python data science world.
Namely, it provides an easy and flexible interface to optimized computation with arrays of data.

Computation on NumPy arrays can be very fast, or it can be very slow.
The key to making it fast is to use **vectorized operations**, generally implemented through NumPy's **universal functions (ufuncs).**


This section motivates the need for NumPy's ufuncs, which can be used to make **repeated calculations** on array elements much more efficient.
It then introduces many of the most common and useful arithmetic ufuncs 

## The Slowness of Loops

Python's default implementation (known as CPython) does some operations very slowly.
This is in part due to the dynamic, interpreted nature of the language: the fact that types are flexible, so that sequences of operations cannot be compiled down to efficient machine code as in languages like C and Fortran.


Recently there have been various attempts to address this weakness: well-known examples are the [PyPy](http://pypy.org/) project, a just-in-time compiled implementation of Python;

The [Cython](http://cython.org) project, which converts Python code to compilable C code; and the [Numba](http://numba.pydata.org/) project, which converts snippets of Python code to fast LLVM bytecode.


Each of these has its strengths and weaknesses, but it is safe to say that none of the three approaches has yet surpassed the reach and popularity of the standard CPython engine.

The relative **sluggishness of Python** generally manifests itself in situations where **many small operations are being repeated** – for instance **looping over arrays** to operate on each element.


For example, imagine we have an array of values and we'd like to compute the reciprocal of each. A straightforward approach might look like this:

In [ ]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

This implementation probably feels fairly natural to someone from, say, a C or Java background.
But if we measure the execution time of this code for a large input, we see that this operation is very slow, perhaps surprisingly so!
We'll benchmark this with IPython's ``%timeit`` magic.

In [17]:
big_array = np.random.randint(1, 100, size=1000000)
%time compute_reciprocals(big_array)

NameError: name 'compute_reciprocals' is not defined

It takes several seconds to compute these million operations and to store the result!


When even cell phones have processing speeds measured in Giga-FLOPS (i.e., billions of numerical operations per second), this seems almost absurdly slow.


It turns out that the **bottleneck** here is not the operations themselves, but the **type-checking** and **function dispatches** that CPython must do at each cycle of the loop.


Each time the reciprocal is computed, Python first examines the object's type and does a dynamic lookup of the correct function to use for that type.
If we were working in compiled code instead, this type specification would be known before the code executes and the result could be computed much more efficiently.

## Introducing UFuncs

For many types of operations, NumPy provides a **convenient interface** into just this kind of statically typed, compiled routine. 

This is known as a **vectorized** operation.


This can be accomplished by simply performing an operation on the array, which will then be applied to each element.


This vectorized approach is designed to **push the loop into the compiled layer that underlies NumPy, leading to much faster execution.**

Compare the results of the following two:

In [ ]:
print(compute_reciprocals(values))
print(1.0 / values)

Looking at the execution time for our big array, we see that it completes orders of magnitude faster than the Python loop:

In [ ]:
%time (1.0 / big_array)

**Vectorized operations** in NumPy are implemented via **ufuncs**, whose main purpose is to quickly execute repeated operations on values in NumPy arrays.


**Ufuncs** are extremely flexible – before we saw an operation between a scalar and an array, but we can also operate between two arrays:

In [ ]:
np.arange(5) / np.arange(1, 6)

And ufunc operations are not limited to one-dimensional arrays–they can also act on multi-dimensional arrays as well:

In [ ]:
x = np.arange(9).reshape((3, 3))
2 ** x

Computations using vectorization through ufuncs are nearly always more efficient than their counterpart implemented using Python loops, especially as the arrays grow in size.


Any time you see such a loop in a Python script, you should consider whether it can be replaced with a vectorized expression.

## Exploring NumPy's UFuncs

Ufuncs exist in two flavors:
- *unary ufuncs*, which operate on a single input.
- *binary ufuncs*, which operate on two inputs.


We'll see examples of both these types of functions here.

### Array arithmetic

NumPy's ufuncs feel very natural to use because they make use of Python's native arithmetic operators.
The standard addition, subtraction, multiplication, and division can all be used:

In [ ]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

There is also a unary ufunc for negation, and a ``**`` operator for exponentiation, and a ``%`` operator for modulus:

In [ ]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

In addition, these can be strung together however you wish, and the standard order of operations is respected:

In [ ]:
-(0.5*x + 1) ** 2

Each of these arithmetic operations are simply **convenient wrappers** around specific functions built into NumPy.


For example, the ``+`` operator is a wrapper for the ``add`` function:

In [ ]:
np.add(x, 2)

The following table lists the arithmetic operators implemented in NumPy:

| Operator	    | Equivalent ufunc    | Description                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Addition (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Subtraction (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Unary negation (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplication (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |Division (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor division (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponentiation (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulus/remainder (e.g., ``9 % 4 = 1``)|

Additionally there are Boolean/bitwise operators.

### Absolute value

Just as NumPy understands Python's built-in arithmetic operators, it also understands Python's built-in absolute value function:

In [ ]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

The corresponding NumPy ufunc is ``np.absolute``, which is also available under the alias ``np.abs``:

In [ ]:
np.absolute(x)

In [ ]:
np.abs(x)

This ufunc can also handle complex data, in which the absolute value returns the magnitude:

In [ ]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

### Trigonometric functions

NumPy provides a large number of useful ufuncs, and some of the most useful for the data scientist are the trigonometric functions.
We'll start by defining an array of angles:

In [ ]:
theta = np.linspace(0, np.pi, 3)
theta

Now we can compute some trigonometric functions on these values:

In [ ]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

The values are computed to within machine precision, which is why values that should be zero do not always hit exactly zero.
Inverse trigonometric functions are also available:

In [ ]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

### Exponents and logarithms

Another common type of operation available in a NumPy ufunc are the exponentials:

In [ ]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

The inverse of the exponentials, the logarithms, are also available.
The basic ``np.log`` gives the natural logarithm; if you prefer to compute the base-2 logarithm or the base-10 logarithm, these are available as well:

In [ ]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

There are also some specialized versions that are useful for maintaining precision with very small input:

In [ ]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

When ``x`` is very small, these functions give more precise values than if the raw ``np.log`` or ``np.exp`` were to be used.

### Specialized ufuncs

NumPy has many more ufuncs available, including hyperbolic trig functions, bitwise arithmetic, comparison operators, conversions from radians to degrees, rounding and remainders, and much more.
A look through the NumPy documentation reveals a lot of interesting functionality.

Another excellent source for more specialized and obscure ufuncs is the submodule ``scipy.special``.
If you want to compute some obscure mathematical function on your data, chances are it is implemented in ``scipy.special``.
There are far too many functions to list them all, but the following snippet shows a couple that might come up in a statistics context:

In [ ]:
from scipy import special

In [ ]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

In [ ]:
# Error function (integral of Gaussian)
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

There are many, many more ufuncs available in both NumPy and ``scipy.special``.
Because the documentation of these packages is available online, a web search along the lines of "gamma function python" will generally find the relevant information.

## Advanced Ufunc Features

Many NumPy users make use of ufuncs without ever learning their full set of features.
We'll outline a few specialized features of ufuncs here.

### Specifying output

For large calculations, it is sometimes useful to be able to specify the array where the result of the calculation will be stored.
Rather than creating a temporary array, this can be used to write computation results directly to the memory location where you'd like them to be.
For all ufuncs, this can be done using the ``out`` argument of the function:

In [ ]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

This can even be used with array views. For example, we can write the results of a computation to every other element of a specified array:

In [ ]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

If we had instead written ``y[::2] = 2 ** x``, this would have resulted in the creation of a temporary array to hold the results of ``2 ** x``, followed by a second operation copying those values into the ``y`` array.
This doesn't make much of a difference for such a small computation, but for very large arrays the memory savings from careful use of the ``out`` argument can be significant.

### Aggregates

For binary ufuncs, there are some interesting aggregates that can be computed directly from the object.
For example, if we'd like to *reduce* an array with a particular operation, we can use the ``reduce`` method of any ufunc.
A reduce repeatedly applies a given operation to the elements of an array until only a single result remains.

For example, calling ``reduce`` on the ``add`` ufunc returns the sum of all elements in the array:

In [ ]:
x = np.arange(1, 6)
np.add.reduce(x)

Similarly, calling ``reduce`` on the ``multiply`` ufunc results in the product of all array elements:

In [ ]:
np.multiply.reduce(x)

If we'd like to store all the intermediate results of the computation, we can instead use ``accumulate``:

In [ ]:
np.add.accumulate(x)

In [ ]:
np.multiply.accumulate(x)

Note that for these particular cases, there are dedicated NumPy functions to compute the results (``np.sum``, ``np.prod``, ``np.cumsum``, ``np.cumprod``), which we'll explore later. 

### Outer products

Finally, any ufunc can compute the output of all pairs of two different inputs using the ``outer`` method.
This allows you, in one line, to do things like create a multiplication table:

x = np.arange(1, 6)
np.multiply.outer(x, x)

The ``ufunc.at`` and ``ufunc.reduceat`` methods, which we'll explore in [Fancy Indexing](02.07-Fancy-Indexing.ipynb), are very helpful as well.

Another extremely useful feature of ufuncs is the ability to operate between arrays of different sizes and shapes, a set of operations known as *broadcasting*.
This subject is important enough that we will devote a whole section to it later. 

# Aggregations: Min, Max, and Everything In Between
Often when faced with a large amount of data, a first step is to compute summary statistics for the data in question.
Perhaps the most common summary statistics are the mean and standard deviation, which allow you to summarize the "typical" values in a dataset, but other aggregates are useful as well (the sum, product, median, minimum and maximum, quantiles, etc.).

NumPy has fast built-in aggregation functions for working on arrays; we'll discuss and demonstrate some of them here.

## Summing the Values in an Array

As a quick example, consider computing the sum of all values in an array.
Python itself can do this using the built-in ``sum`` function:

In [ ]:
import numpy as np

In [ ]:
L = np.random.random(100)
sum(L)

The syntax is quite similar to that of NumPy's ``sum`` function, and the result is the same in the simplest case:

In [ ]:
np.sum(L)

However, because it executes the operation in compiled code, NumPy's version of the operation is computed much more quickly:

In [ ]:
big_array = np.random.rand(1000000)
%timeit sum(big_array)
%timeit np.sum(big_array)

Be careful, though: the ``sum`` function and the ``np.sum`` function are not identical, which can sometimes lead to confusion!
In particular, their optional arguments have different meanings, and ``np.sum`` is aware of multiple array dimensions, as we will see in the following section.

## Minimum and Maximum

Similarly, Python has built-in ``min`` and ``max`` functions, used to find the minimum value and maximum value of any given array:

In [ ]:
min(big_array), max(big_array)

NumPy's corresponding functions have similar syntax, and again operate much more quickly:

In [ ]:
np.min(big_array), np.max(big_array)

In [ ]:
%timeit min(big_array)
%timeit np.min(big_array)

For ``min``, ``max``, ``sum``, and several other NumPy aggregates, a shorter syntax is to use methods of the array object itself:

In [ ]:
print(big_array.min(), big_array.max(), big_array.sum())

Whenever possible, make sure that you are using the NumPy version of these aggregates when operating on NumPy arrays!

### Multi dimensional aggregates

One common type of aggregation operation is an aggregate along a row or column.
Say you have some data stored in a two-dimensional array:

In [ ]:
M = np.random.random((3, 4))
print(M)

By default, each NumPy aggregation function will return the aggregate over the entire array:

In [ ]:
M.sum()

Aggregation functions take an additional argument specifying the *axis* along which the aggregate is computed. For example, we can find the minimum value within each column by specifying ``axis=0``:

In [ ]:
M.min(axis=0)

The function returns four values, corresponding to the four columns of numbers.

Similarly, we can find the maximum value within each row:

In [ ]:
M.max(axis=1)

The way the axis is specified here can be confusing to users coming from other languages.
The ``axis`` keyword specifies the *dimension of the array that will be collapsed*, rather than the dimension that will be returned.
So specifying ``axis=0`` means that the first axis will be collapsed: for two-dimensional arrays, this means that values within each column will be aggregated.

### Other aggregation functions

NumPy provides many other aggregation functions, but we won't discuss them in detail here.
Additionally, most aggregates have a ``NaN``-safe counterpart that computes the result while ignoring missing values, which are marked by the special IEEE floating-point ``NaN`` value.
Some of these ``NaN``-safe functions were not added until NumPy 1.8, so they will not be available in older NumPy versions.

The following table provides a list of useful aggregation functions available in NumPy:

|Function Name      |   NaN-safe Version  | Description                                   |
|-------------------|---------------------|-----------------------------------------------|
| ``np.sum``        | ``np.nansum``       | Compute sum of elements                       |
| ``np.prod``       | ``np.nanprod``      | Compute product of elements                   |
| ``np.mean``       | ``np.nanmean``      | Compute mean of elements                      |
| ``np.std``        | ``np.nanstd``       | Compute standard deviation                    |
| ``np.var``        | ``np.nanvar``       | Compute variance                              |
| ``np.min``        | ``np.nanmin``       | Find minimum value                            |
| ``np.max``        | ``np.nanmax``       | Find maximum value                            |
| ``np.argmin``     | ``np.nanargmin``    | Find index of minimum value                   |
| ``np.argmax``     | ``np.nanargmax``    | Find index of maximum value                   |
| ``np.median``     | ``np.nanmedian``    | Compute median of elements                    |
| ``np.percentile`` | ``np.nanpercentile``| Compute rank-based statistics of elements     |
| ``np.any``        | N/A                 | Evaluate whether any elements are true        |
| ``np.all``        | N/A                 | Evaluate whether all elements are true        |

We will see these aggregates often throughout this module. 

## Example: What is the Average Height of US Presidents?

Aggregates available in NumPy can be extremely useful for summarizing a set of values.
As a simple example, let's consider the heights of all US presidents.
This data is available in the file *president_heights.csv*, which is a simple comma-separated list of labels and values:

In [ ]:
!head -4 data/president_heights.csv

We'll use the Pandas package, (which we'll explore more fully later) to read the file and extract this information (note that the heights are measured in centimeters).

In [ ]:
import pandas as pd
data = pd.read_csv('data/president_heights.csv')
heights = np.array(data['height(cm)'])
print(heights)

Now that we have this data array, we can compute a variety of summary statistics:

In [ ]:
print("Mean height:       ", heights.mean())
print("Standard deviation:", heights.std())
print("Minimum height:    ", heights.min())
print("Maximum height:    ", heights.max()) 

Note that in each case, the aggregation operation reduced the entire array to a single summarizing value, which gives us information about the distribution of values.
We may also wish to compute quantiles:

In [ ]:
print("25th percentile:   ", np.percentile(heights, 25))
print("Median:            ", np.median(heights))
print("75th percentile:   ", np.percentile(heights, 75))

We see that the median height of US presidents is 182 cm, or just shy of six feet.

Of course, sometimes it's more useful to see a visual representation of this data, which we can accomplish using tools in Matplotlib (we'll discuss Matplotlib more later ). For example, this code generates the following chart:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # set plot style

In [ ]:
plt.hist(heights)
plt.title('Height Distribution of US Presidents')
plt.xlabel('height (cm)')
plt.ylabel('number');

These aggregates are some of the fundamental pieces of exploratory data analysis that we'll explore in more depth in later.

# GOOD LUCK!